# Docker-Python "Swarm Mode" Lab

Based on Mario's gist here: https://gist.github.com/l0rd/5186cc80f8f26dc7e9490abca4405830

In [1]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

# Requirements
- Docker >= 1.12 (docker-17.05-ce recommended)
- Docker machine
- Virtualbox

# Setup

## Install docker and docker-machine modules

## NOTE:
#### python-docker-machine installed from https://github.com/mjbright/python-docker-machine

Simple modification to depend upon 'docker' not 'docker-py'.

Installed using

'''python setup.py install'''


## Check that docker is installed, but not docker-py !!

In [2]:
!which pip
!pip list --format=columns | grep -i docker

!pip list --format=columns | grep docker-py || echo -e "\n\n**** ERROR: you have Docker-py installed!!"

/home/mjb/usr/anaconda3/bin/pip
docker                             2.2.1      
docker-pycreds                     0.2.1      
python-docker-machine              0.2.2      
docker-pycreds                     0.2.1      


In [3]:
from distutils.version import LooseVersion
import docker

EXPECT_DOCKER_VERSION='2.2.1'

print("docker module version=" + docker.version)

if LooseVersion(docker.version) < LooseVersion(EXPECT_DOCKER_VERSION):
    print("\n\nERROR: Bad docker.py version: {} < expected {}".format(docker.version, EXPECT_DOCKER_VERSION))
else:
    print("\n\nSUCCESS: you're all set to continue this Lab!")
  

docker module version=2.2.1


# Creating a Docker client connection to the Docker engine

During this lab we will use the installed Docker Client to talk to your Docker hosts.

Those hosts may be running 
- locally      controlled via docker-machine
- in the cloud controlled via docker-machine
- under Docker Toolbox controlled via docker-machine (https://docs.docker.com/machine/get-started/#create-a-machine)
- under Docker For Windows, controlled via docker-machine (https://docs.docker.com/docker-for-windows/#docker-settings)
- Docker for Mac, where you will only be able to have a single-node swarm (unless bursting out to cloud)

This notebook demonstrates the use of docker-machine to control the hosts.

#### NOTE: The following command is specific to my environment as I have a special DOCKER_HOST value set:

## Connection to a local Docker host

#### NOTE: This is for information, in our lab we will obtain a client connection via the python-docker-machine module, see below.

In [5]:
# NOTE: specific to my environment
#       if you have some special DOCKER_HOST setting, then check this and set in the Python code below
#!. ~/.docker.rc; ! echo $DOCKER_HOST

#### NOTE: You may comment the 'DOCKER_HOST' lines below if you have no special configuration:

In [6]:
import docker, os

# Open a client connection to your local Docker host, set DOCKER_HOST if you have special settings to use:

# Use your DOCKER_HOST value here:
# DOCKER_HOST='tcp://127.0.0.1:2475'

# os.environ['DOCKER_HOST']=DOCKER_HOST

# auto: use same api version as docker daemon
client = docker.from_env(version='auto') #, url='tcp://127.0.0.1:2475')

# Using python-docker-machine wrapper module

Here we will describe use of this Python module which provides Python access to the 'docker-machine' executable on your machine

### Referencing the docker-machine executable

In [7]:
import machine
import docker

m = machine.Machine(path="/usr/local/bin/docker-machine")

In [8]:
print(m)

### Cleanup:  Delete any existing machines

In [9]:
# If you want to delete existing machines:
# !docker-machine rm -f swmaster swnode1 swnode2

DELETE_MACHINES=True
if DELETE_MACHINES:
    for node in m.ls():
        if node['Name'] != '':
            print("Deleting node " + node['Name'])
            m.rm(machine=node['Name'],force=True)
            
m.ls()

Deleting node swmaster1
Deleting node swnode1
Deleting node swnode2


[{'Name': ''}]

### Creating a new machine

If creating a machine in some cloud provider you will need to first export some appropriate token or subscription id as
an appropriately named environment variable, e.g.
- DIGITALOCEAN_ACCESS_TOKEN for DigitalOcean
- AZURE_SUBSCRIPTION_ID for Microsoft Azure

Refer to https://docs.docker.com/machine/get-started-cloud/ for information,

and to https://docs.docker.com/machine/drivers/gce/ (select cloud provider in left-hand sidebar) for specifics.

**WARNING**: You may need to "logon" to the platform before using the python-docker-machine otherwise the python-docker-machine module will block asking you to validate - but **you will not see that message** !
** This will not work with Azure ** because of the need to logon from the local terminal

## Examples:

### Digital Ocean:  
Set DIGITALOCEAN_ACCESS_TOKEN as an environment variable 

e.g.
```
     os.environ['DIGITALOCEAN_ACCESS_TOKEN']='MY-TOKEN'
     m.create('swmaster1', driver='digitalocean', blocking=True)
```

### Azure:
Set AZURE_SUBSCRIPTION_ID as an environment variable 

e.g.
```
    os.environ['AZURE_SUBSCRIPTION_ID']='MY-ID'
    m.create('swmaster1', driver='azure', blocking=True)
```

### Google Compute Engine:
Use "gcloud auth login"

e.g.
```
     m.create('swmaster1', driver='google', blocking=True)
```

## Example of local machine creation

Below is an example of machine creation on a local machine, to be adapted as above to run in the cloud

In [10]:
m.create('test-machine', driver='virtualbox', blocking=True)

m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'test-machine',
  'ResponseTime': '696ms',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Name': ''}]

## Deletion of a local machine

In [12]:
!docker-machine ls

NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS


In [11]:
m.rm(machine='test-machine')

m.ls()

[{'Name': ''}]

In [12]:
!docker-machine ls

NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS


# Create nodes for your swarm cluster (1 master and 2 worker nodes)

We will create 3 nodes using docker-machine/virtualbox.

In [13]:
#### If not already created, we can create our machines here:

import datetime
import time
import os

def createNodes(nodes):
    for node in nodes:
        NOW=datetime.datetime.now().strftime("%H:%M:%S")
        SECS_BEFORE=int(time.time())
        print("{} Creating node {} <on {}> ...".format(NOW, node, MACHINE_DRIVER))
        m.create(node, driver=MACHINE_DRIVER, blocking=False)
        SECS_AFTER=int(time.time())
        print("... took {} secs".format(SECS_AFTER-SECS_BEFORE))


# FAILS on AZURE due to interactive validation required !
#MACHINE_DRIVER='azure'
#os.environ['AZURE_SUBSCRIPTION_ID']='azure-sub-id'

# MACHINE_DRIVER='digitalocean'
#os.environ['DIGITALOCEAN_ACCESS_TOKEN']='my-token'

MACHINE_DRIVER='virtualbox'

MASTER_NODES=['swmaster1']
WORKER_NODES=['swnode1','swnode2']

#MASTER_NODES=['swmaster1','swmaster2','swmaster3']
#WORKER_NODES=['swnode1','swnode2']

In [15]:
CREATE_MACHINES=True

if CREATE_MACHINES:
    createNodes(MASTER_NODES + WORKER_NODES)

13:31:41 Creating node swmaster1 <on virtualbox> ...
... took 106 secs
13:33:27 Creating node swnode1 <on virtualbox> ...
... took 105 secs
13:35:12 Creating node swnode2 <on virtualbox> ...
... took 108 secs


#### NOTE: Temporary errors from "*docker-machine ls*"

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

![](images/docker-machine-errors.png)

## Recovering from possible docker-machine/VirtualBox problems.

It's possible that this machine creation fails, e.g. with error as below:

```
RuntimeError: cmd returned error 1: Error creating machine: Error in driver during machine creation: /usr/bin/VBoxManage modifyvm swmaster1 --natpf1 ssh,tcp,127.0.0.1,40886,,22 failed:
VBoxManage: error: The machine 'swmaster1' is already locked for a session (or being unlocked)
VBoxManage: error: Details: code VBOX_E_INVALID_OBJECT_STATE (0x80bb0007), component MachineWrap, interface IMachine, callee nsISupports
VBoxManage: error: Context: "LockMachine(a->session, LockType_Write)" at line 493 of file VBoxManageModifyVM.cpp
```

To recover, list current VMs known by VirtualBox using
```VBoxManage list vms```

For any VMs which are marked as 'inaccessible', delete them using their ID, e.g.
```VBoxManage unregistervm 576f28c7-796e-47d4-a1b1-55dada4bba14```

If other machines are listed in error seen by
```docker-machine ls```

try to delete theme with
```docker-machine rm -f <machine-name>```

If this fails, perform a "VBoxManage unregistervm", of the VM id or it's name
e.g.
```VBoxManage unregistervm 576f28c7-796e-47d4-a1b1-55dada4bba14```
or
```VBoxManage unregistervm swmaster1```

Then try to create machines again

In [16]:
m.ls()

[{'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swmaster1',
  'ResponseTime': '1.748s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.100:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode1',
  'ResponseTime': '1.793s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.101:2376'},
 {'Active': '-',
  'ActiveHost': 'false',
  'ActiveSwarm': 'false',
  'DockerVersion': 'v17.05.0-ce',
  'DriverName': 'virtualbox',
  'Error': '',
  'Name': 'swnode2',
  'ResponseTime': '1.747s',
  'State': 'Running',
  'Swarm': '',
  'URL': 'tcp://192.168.99.102:2376'},
 {'Name': ''}]

We can obtain the "config" parameters (o/p of "docker-machine config <machine name>")

In [17]:
m.config(machine='swmaster1')

{'base_url': 'https://192.168.99.100:2376',
 'tls': <docker.tls.TLSConfig at 0x7f1b9f61c0b8>}

We can obtain the "env" parameters (o/p of "docker-machine env <machine name>")

In [20]:
m.env(machine='swmaster1')

['export',
 'DOCKER_TLS_VERIFY="1"',
 'export',
 'DOCKER_HOST="tcp://192.168.99.100:2376"',
 'export',
 'DOCKER_CERT_PATH="/home/mjb/.docker/machine/machines/swmaster1"',
 'export',
 'DOCKER_MACHINE_NAME="swmaster1"',
 '#',
 'Run',
 'this',
 'command',
 'to',
 'configure',
 'your',
 'shell:',
 '#',
 'eval',
 '$(/usr/local/bin/docker-machine',
 'env',
 'swmaster1)']

Now let's create docker-py client objects for each of our new nodes

In [21]:
# Create Docker clients for each machine:

nodes={}

swmaster1 = docker.DockerClient(**m.config(machine='swmaster1'))
nodes['swmaster1']=swmaster1

# For rich people with more nodes ...
# swmaster2 = docker.DockerClient(**m.config(machine='swmaster2'))
# nodes['swmaster2']=swmaster2
# 
# swmaster3 = docker.DockerClient(**m.config(machine='swmaster3'))
# nodes['swmaster3']=swmaster3

swnode1 = docker.DockerClient(**m.config(machine='swnode1'))
nodes['swnode1']=swnode1

swnode2 = docker.DockerClient(**m.config(machine='swnode2'))
nodes['swnode2']=swnode2

pp.pprint(nodes)

{   'swmaster1': <docker.client.DockerClient object at 0x7f1b9f535160>,
    'swnode1': <docker.client.DockerClient object at 0x7f1b9f601160>,
    'swnode2': <docker.client.DockerClient object at 0x7f1b9f60f400>}


In [23]:
# Check connectivity to machines:

print("Checking that nodes are 'ping'able")

for node in nodes.keys():
    #print(node)
    print("{}: {}".format(node, nodes[node].ping()))

#print(swmaster.ping(), swnode1.ping(), swnode2.ping())

Checking that nodes are 'ping'able
swmaster1: True
swnode1: True
swnode2: True


In [24]:
swmaster1.images.list()

[]

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.


Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster*'.

### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [25]:

def show_networks(client):
    FORMAT_STR="%-14s %-20s %-10s %-14s"
    print(FORMAT_STR % ("NETWORK ID", "NAME", "DRIVER", "SCOPE"))

    for network in client.networks.list():
        #print(dir(network))
        print(FORMAT_STR % (network.short_id, network.name, network.attrs['Driver'], network.attrs['Scope']))
        
        
# NETWORK ID          NAME                DRIVER              SCOPE
# ab973c7b9062        bridge              bridge              local
# e953fc8fe3f2        host                host                local
# d45eb5315d7c        none                null                local

show_networks(swmaster1)

NETWORK ID     NAME                 DRIVER     SCOPE         
8172ddeaf4     host                 host       local         
ae1f4f5c92     none                 null       local         
6b36ae0d67     bridge               bridge     local         


### Obtain ip addresses of nodes, esp. the 1st master node

Now let's identify the ip address of our nodes.

We need to note the ip address of the master node, so that when we initialize our swarm cluster, this node will advertise itself on this address so that other nodes can join the swarm cluster.

We can see this through config or ip commands of docker-machine as shown below.

In [26]:
for node in nodes.keys():
    ip = m.ip(machine=node)
    print("{}: {}".format(node, ip))

    
master1ip = m.ip(machine='swmaster1')

swmaster1: 192.168.99.100
swnode1: 192.168.99.101
swnode2: 192.168.99.102


We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster".

The following command will run swarm init to generate the cluster with 'swmaster' as the Master node.
You should see output similar to the below:

In [27]:
swmaster1.info()

{'Architecture': 'x86_64',
 'BridgeNfIp6tables': True,
 'BridgeNfIptables': True,
 'CPUSet': True,
 'CPUShares': True,
 'CgroupDriver': 'cgroupfs',
 'ClusterAdvertise': '',
 'ClusterStore': '',
 'ContainerdCommit': {'Expected': '9048e5e50717ea4497b757314bad98ea3763c145',
  'ID': '9048e5e50717ea4497b757314bad98ea3763c145'},
 'Containers': 0,
 'ContainersPaused': 0,
 'ContainersRunning': 0,
 'ContainersStopped': 0,
 'CpuCfsPeriod': True,
 'CpuCfsQuota': True,
 'Debug': True,
 'DefaultRuntime': 'runc',
 'DockerRootDir': '/mnt/sda1/var/lib/docker',
 'Driver': 'aufs',
 'DriverStatus': [['Root Dir', '/mnt/sda1/var/lib/docker/aufs'],
  ['Backing Filesystem', 'extfs'],
  ['Dirs', '0'],
  ['Dirperm1 Supported', 'true']],
 'ExperimentalBuild': False,
 'HttpProxy': '',
 'HttpsProxy': '',
 'ID': 'JOQ2:PER3:G43N:FHNJ:BMI4:6JCE:CEG6:HDV6:PRBK:FBPB:ONJH:AIRY',
 'IPv4Forwarding': True,
 'Images': 0,
 'IndexServerAddress': 'https://index.docker.io/v1/',
 'InitBinary': 'docker-init',
 'InitCommit': {'Ex

In [28]:
swmaster1.info()['Swarm']

{'ControlAvailable': False,
 'Error': '',
 'LocalNodeState': 'inactive',
 'NodeAddr': '',
 'NodeID': '',
 'RemoteManagers': None}

### Before creating the swarm cluster we have 0 nodes, and an inactive cluster

In [29]:
if 'Nodes' in swmaster1.info()['Swarm']:
    swmaster1.info()['Swarm']['Nodes']

In [30]:
swmaster1.info()['Swarm']['LocalNodeState']

'inactive'

In [32]:
# Note: the following is specific to the Jupyter notebook environment allowing to introspect class/method/variable definitions:

swmaster1.swarm.init?

Now let's initialize our swarm mode.

This make this node a 'Manager' node of the cluster and creates new overlay network to be used by services running in the cluster.

In [33]:
swmaster1.swarm.init(advertise_addr=master1ip)

In [34]:
swarm_info = swmaster1.swarm.client.info()

pp.pprint(swarm_info)

{   'Architecture': 'x86_64',
    'BridgeNfIp6tables': True,
    'BridgeNfIptables': True,
    'CPUSet': True,
    'CPUShares': True,
    'CgroupDriver': 'cgroupfs',
    'ClusterAdvertise': '',
    'ClusterStore': '',
    'ContainerdCommit': {   'Expected': '9048e5e50717ea4497b757314bad98ea3763c145',
                            'ID': '9048e5e50717ea4497b757314bad98ea3763c145'},
    'Containers': 0,
    'ContainersPaused': 0,
    'ContainersRunning': 0,
    'ContainersStopped': 0,
    'CpuCfsPeriod': True,
    'CpuCfsQuota': True,
    'Debug': True,
    'DefaultRuntime': 'runc',
    'DockerRootDir': '/mnt/sda1/var/lib/docker',
    'Driver': 'aufs',
    'DriverStatus': [   ['Root Dir', '/mnt/sda1/var/lib/docker/aufs'],
                        ['Backing Filesystem', 'extfs'],
                        ['Dirs', '0'],
                        ['Dirperm1 Supported', 'true']],
    'ExperimentalBuild': False,
    'HttpProxy': '',
    'HttpsProxy': '',
    'ID': 'JOQ2:PER3:G43N:FHNJ:BMI4:6JCE:CEG6

### After creating the swarm cluster we have 1 nodes, and an active cluster

In [35]:
swmaster1.info()['Swarm']['Nodes']

1

A docker info should now show "Swarm: active" as below:

In [36]:
swmaster1.info()['Swarm']['LocalNodeState']

'active'

In [37]:
pp.pprint(swmaster1.swarm.attrs)

{   'CreatedAt': '2017-05-11T11:43:05.297040351Z',
    'ID': 'p6xdvnj76fop1xqgox7qxys1z',
    'JoinTokens': {   'Manager': 'SWMTKN-1-4x9d5aan2d32e9w86ao991b39b82gq65u82hcar1t2zd5ledfx-cu6udyyfxmsfuzgcnyhe2kyy6',
                      'Worker': 'SWMTKN-1-4x9d5aan2d32e9w86ao991b39b82gq65u82hcar1t2zd5ledfx-9bscsxf5f4ek7v6737tuh9uzs'},
    'Spec': {   'CAConfig': {'NodeCertExpiry': 7776000000000000},
                'Dispatcher': {'HeartbeatPeriod': 5000000000},
                'EncryptionConfig': {'AutoLockManagers': False},
                'Labels': {},
                'Name': 'default',
                'Orchestration': {'TaskHistoryRetentionLimit': 5},
                'Raft': {   'ElectionTick': 3,
                            'HeartbeatTick': 1,
                            'KeepOldSnapshots': 0,
                            'LogEntriesForSlowFollowers': 500,
                            'SnapshotInterval': 10000},
                'TaskDefaults': {}},
    'UpdatedAt': '2017-05-11T11:43:05.

### We can obtain the join tokens allowing to join new nodes as manager or worker nodes

In [38]:
join_tokens = swmaster1.swarm.attrs['JoinTokens']

pp.pprint(join_tokens)

manager_token = join_tokens['Manager']
worker_token = join_tokens['Worker']

{   'Manager': 'SWMTKN-1-4x9d5aan2d32e9w86ao991b39b82gq65u82hcar1t2zd5ledfx-cu6udyyfxmsfuzgcnyhe2kyy6',
    'Worker': 'SWMTKN-1-4x9d5aan2d32e9w86ao991b39b82gq65u82hcar1t2zd5ledfx-9bscsxf5f4ek7v6737tuh9uzs'}


If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [40]:
show_networks(swmaster1)
#show_networks(swmaster2)

NETWORK ID     NAME                 DRIVER     SCOPE         
8fhgsmcv2i     ingress              overlay    swarm         
8172ddeaf4     host                 host       local         
ae1f4f5c92     none                 null       local         
6b36ae0d67     bridge               bridge     local         
4124e7c3a7     docker_gwbridge      bridge     local         


# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

In [42]:
print(manager_token)

SWMTKN-1-4x9d5aan2d32e9w86ao991b39b82gq65u82hcar1t2zd5ledfx-cu6udyyfxmsfuzgcnyhe2kyy6


In [43]:
pp.pprint(swmaster1.info()['Swarm'])

{   'Cluster': {   'CreatedAt': '2017-05-11T11:43:05.297040351Z',
                   'ID': 'p6xdvnj76fop1xqgox7qxys1z',
                   'Spec': {   'CAConfig': {'NodeCertExpiry': 7776000000000000},
                               'Dispatcher': {'HeartbeatPeriod': 5000000000},
                               'EncryptionConfig': {'AutoLockManagers': False},
                               'Labels': {},
                               'Name': 'default',
                               'Orchestration': {   'TaskHistoryRetentionLimit': 5},
                               'Raft': {   'ElectionTick': 3,
                                           'HeartbeatTick': 1,
                                           'KeepOldSnapshots': 0,
                                           'LogEntriesForSlowFollowers': 500,
                                           'SnapshotInterval': 10000},
                               'TaskDefaults': {}},
                   'UpdatedAt': '2017-05-11T11:43:05.953367677Z',
   

In [ ]:
print(master1ip)

## Add 2nd, 2rd master nodes if present

In [44]:
#if swmaster2 != None:
#    swmaster2.swarm.join(join_token=manager_token, remote_addrs=[master1ip+':2377'],
#                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

In [45]:
#if swmaster3 != None:
#    swmaster3.swarm.join(join_token=manager_token, remote_addrs=[master1ip+':2377'],
#                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

In [46]:
!. ~/.docker.rc; docker $(docker-machine config swmaster1) node list

/bin/sh: 1: .: Can't open /home/mjb/.docker.rc


In [47]:
def show_nodes(client):
    FORMAT_STR="%-14s %-10s %-10s %-14s %-10s"
    print(FORMAT_STR % ("ID", "HOSTNAME", "STATE", "AVAILABILITY", "MANAGER STATUS"))
    
    for node in client.nodes.list():
        #pp.pprint(node.attrs)
        nodeSwarmStatus=''
        state = node.attrs['Status']['State']
        if 'ManagerStatus' in node.attrs:
            if 'Leader' in node.attrs['ManagerStatus'] and node.attrs['ManagerStatus']['Leader']:
                nodeSwarmStatus='leader'
            elif 'Reachability' in node.attrs['ManagerStatus']:
                nodeSwarmStatus=node.attrs['ManagerStatus']['Reachability']
        print(FORMAT_STR % (node.id[:12], node.attrs['Description']['Hostname'], state, node.attrs['Spec']['Availability'], nodeSwarmStatus))

        
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
hyg7lucp604t   swmaster1  ready      active         leader    


## Add worker nodes

In [48]:
print(worker_token)

SWMTKN-1-4x9d5aan2d32e9w86ao991b39b82gq65u82hcar1t2zd5ledfx-9bscsxf5f4ek7v6737tuh9uzs


Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [49]:
#print( "swmaster.nodes()=" + str( swmaster.nodes() ))
#print()

import json

json_nodes = swmaster1.nodes.list()

#print(type(json_nodes))

for node in json_nodes:
    pp.pprint("node {}.attrs=".format(node.attrs['Description']['Hostname']))
    pp.pprint(node.attrs)
    print()
    

'node swmaster1.attrs='
{   'CreatedAt': '2017-05-11T11:43:05.297289966Z',
    'Description': {   'Engine': {   'EngineVersion': '17.05.0-ce',
                                     'Labels': {'provider': 'virtualbox'},
                                     'Plugins': [   {   'Name': 'bridge',
                                                        'Type': 'Network'},
                                                    {   'Name': 'host',
                                                        'Type': 'Network'},
                                                    {   'Name': 'macvlan',
                                                        'Type': 'Network'},
                                                    {   'Name': 'null',
                                                        'Type': 'Network'},
                                                    {   'Name': 'overlay',
                                                        'Type': 'Network'},
                                  

In [50]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
hyg7lucp604t   swmaster1  ready      active         leader    


In [51]:
swmaster1.nodes.list()

[<Node: hyg7lucp60>]

In [52]:
# Note: the following is specific to the Jupyter notebook environment allowing to introspect class/method/variable definitions:

swnode1.swarm.join?

In [53]:
swnode1.swarm.join(join_token=worker_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

True

In [54]:
swmaster1.nodes.list()

[<Node: hyg7lucp60>, <Node: i3m48j05r1>]

In [55]:
!docker-machine ls

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.100:2376           v17.05.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.101:2376           v17.05.0-ce   
swnode2     -        virtualbox   Running   tcp://192.168.99.102:2376           v17.05.0-ce   


In [57]:
!docker $(docker-machine config swmaster1) node ls

ID                           HOSTNAME   STATUS  AVAILABILITY  MANAGER STATUS
hyg7lucp604tvmgns77bo2a0i *  swmaster1  Ready   Active        Leader
i3m48j05r1hkyypm9nnrbdyaw    swnode1    Ready   Active        


In [58]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
hyg7lucp604t   swmaster1  ready      active         leader    
i3m48j05r1hk   swnode1    ready      active                   


In [59]:
swnode2.swarm.join(join_token=worker_token, remote_addrs=[master1ip+':2377'],
                  listen_addr='0.0.0.0:5000', advertise_addr='eth0:5000')

True

In [61]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
g3evkow9ylaq   swnode2    ready      active                   
hyg7lucp604t   swmaster1  ready      active         leader    
i3m48j05r1hk   swnode1    ready      active                   


In [62]:
!docker $(docker-machine config swmaster1) node ls

ID                           HOSTNAME   STATUS  AVAILABILITY  MANAGER STATUS
g3evkow9ylaq10z45nyic8x06    swnode2    Ready   Active        
hyg7lucp604tvmgns77bo2a0i *  swmaster1  Ready   Active        Leader
i3m48j05r1hkyypm9nnrbdyaw    swnode1    Ready   Active        


### Need to force 'leave' to remove managers from cluster

## NOTE: dont remove swarm master nodes, this seems to break cluster ... currently

In [63]:
for node in swmaster1.nodes.list():
    pp.pprint(node.attrs['Status'])

{'Addr': '192.168.99.102', 'State': 'ready'}
{'Addr': '192.168.99.100', 'State': 'ready'}
{'Addr': '192.168.99.101', 'State': 'ready'}


In [64]:
# Just for info - DON'T DO THIS (will prevent service creation from working - bug)

# swmaster2.swarm.leave(force=True)
# swmaster3.swarm.leave(force=True)

In [65]:
show_nodes(swmaster1)

ID             HOSTNAME   STATE      AVAILABILITY   MANAGER STATUS
g3evkow9ylaq   swnode2    ready      active                   
hyg7lucp604t   swmaster1  ready      active         leader    
i3m48j05r1hk   swnode1    ready      active                   


In [66]:
# Just for info - DON'T DO THIS (will prevent service creation from working - bug)

# m.rm(machine='swmaster2',force=True)
# m.rm(machine='swmaster3',force=True)

### NOTE: incorrect status

Our show_nodes method shows the managers are unreachable but still active and not necessarily down.

See below, this seems to be a problem in the API, or docker module implementation:

In [67]:
for node in swmaster1.nodes.list():
    pp.pprint(node.attrs['Status'])

{'Addr': '192.168.99.102', 'State': 'ready'}
{'Addr': '192.168.99.100', 'State': 'ready'}
{'Addr': '192.168.99.101', 'State': 'ready'}


# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [68]:
swmaster1.services.list()

[]

Now we will create a new service based on the docker image mariolet/docker-demo

We will expose this service on port 8080


In [69]:
swmaster1.services.create??

In [70]:
docker.types.ContainerSpec?

## Create a service

Now let us use the Client API to create a new service

Note how we create ContainerSpec and TaskTemplate types wich we pass to the Client.Services.create() method

Here is a first version, which launches an alpine image which runs for 60 seconds:

In [71]:
IMAGE = 'alpine'

container_spec = docker.types.ContainerSpec(
    IMAGE, ['echo', 'hello']
)

"""
endpoint_spec = docker.types.EndpointSpec(ports={
    12357: (1990, 'udp'),
    12562: (678,),
    53243: 8080,
})
"""
endpoint_spec = docker.types.EndpointSpec(ports={
    8080: 8080,
})

task_tmpl = docker.types.TaskTemplate(container_spec)

In [72]:
swmaster1.services.create??

In [73]:
#swmaster1.services.create(task_tmpl, name='docker-demo', endpoint_spec=endpoint_spec)

service = swmaster1.services.create(
            # create arguments
            name='alpine-test',
            labels={'foo': 'bar'},
            # ContainerSpec arguments
            image=IMAGE,
            command="sleep 60",
            container_labels={'container': 'label'}
        )

In [75]:
print( swmaster1.services.list() )

!echo "Sleeping ..."; sleep 70

Sleeping ...


after 1 min ...

We might expect that after 1 minute, the service is no longer running.

But in fact adter 1 minute although the container running the sleep command stops the service is still running and recognizing that the container exited it will restart a new container ("*task*") for this service


In [76]:
print( swmaster1.services.list() )


[<Service: o14xis3t7o>]


In [80]:
service = swmaster1.services.list()[0]
print("Service=" + str(service) )
service_id = service.id

print("\nService attributes=" + str(service.attrs) )

print("\nTasks associated with the service:")
for task in service.tasks():
    print("\n---- Task:")
    pp.pprint(task)


Service=<Service: o14xis3t7o>

Service attributes={'ID': 'o14xis3t7o1zll2dqetdmbbs8', 'Version': {'Index': 20}, 'CreatedAt': '2017-05-11T11:50:05.882172836Z', 'UpdatedAt': '2017-05-11T11:50:05.882172836Z', 'Spec': {'Name': 'alpine-test', 'Labels': {'foo': 'bar'}, 'TaskTemplate': {'ContainerSpec': {'Image': 'alpine:latest@sha256:c0537ff6a5218ef531ece93d4984efc99bbf3f7497c0a7726c88e2bb7584dc96', 'Labels': {'container': 'label'}, 'Command': ['sleep', '60']}, 'ForceUpdate': 0}, 'Mode': {'Replicated': {'Replicas': 1}}}, 'Endpoint': {'Spec': {}}}

Tasks associated with the service:

---- Task:
{   'CreatedAt': '2017-05-11T11:53:26.064688844Z',
    'DesiredState': 'shutdown',
    'ID': '50w8m7oeqymga4zqstgltnd9u',
    'Labels': {},
    'NodeID': 'hyg7lucp604tvmgns77bo2a0i',
    'ServiceID': 'o14xis3t7o1zll2dqetdmbbs8',
    'Slot': 1,
    'Spec': {   'ContainerSpec': {   'Command': ['sleep', '60'],
                                     'Image': 'alpine:latest@sha256:c0537ff6a5218ef531ece93d4984

In [81]:
swmaster1.containers.list()

[<Container: 8294b0dba7>]

Now let's remove this service, before creating other services

In [ ]:
service.remove()

In [84]:
print("Running containers on master node:" + str(swmaster1.containers.list()) )

print("Running services on master node:" + str(swmaster1.services.list()))

#swmaster1.services.create(image=IMAGE, name='docker-demo', labels={'label1': 'label2'}, command=None)

Running containers on master node:[]
Running services on master node:[]


# Launching "*docker-demo*" as a service

In [85]:
IMAGE = 'mariolet/docker-demo:20'
IMAGE = 'mjbright/docker-demo:20'

container_spec = docker.types.ContainerSpec(
    IMAGE, ['echo', 'hello']
)

endpoint_spec = docker.types.EndpointSpec(ports={
    8080: 8080,
})


task_tmpl = docker.types.TaskTemplate(container_spec)

docker_demo_service = swmaster1.services.create(
            # create arguments
            name='docker-demo',
            labels={'label1': 'value1'},
            # ContainerSpec arguments
            image=IMAGE,
            #command="sleep 60",
            container_labels={'container': 'label'},
    endpoint_spec=endpoint_spec
        )


In [ ]:
#BUT PORTS NOT SHOWING:
    
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service ps docker-demo

In [ ]:
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service create --replicas 1 --name docker-demo-via-cli -p 9090:8080 mariolet/docker-demo:20

In [ ]:
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service ps docker-demo-via-cli
#!. /home/mjb/.docker.rc; docker $(docker-machine config swmaster1) service rm docker-demo-via-cli

Now we list services again and we should see our newly added docker-demo service

In [86]:
swmaster1.services.list()

[<Service: njmzhx2jig>]

In [88]:
def showServices(master):
    for service in master.services.list():
        print("{}: {}".format(service, service.name))

showServices(swmaster1)

service = swmaster1.services.list()[0]
service_id = service.id

<Service: njmzhx2jig>: docker-demo


In [89]:
#wmaster1.services.get(service_id=service_id).remove()
#howServices(swmaster1)

... and we can look at the service as seen by the cluster:

In [90]:
!docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE           ERROR               PORTS
60ha6jldk92k        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 2 minutes ago                       


... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [93]:
service = swmaster1.services.list()[0]

service.tasks()

[{'CreatedAt': '2017-05-11T11:58:05.371512356Z',
  'DesiredState': 'running',
  'ID': '60ha6jldk92kfnchom2jqqksm',
  'Labels': {},
  'NetworksAttachments': [{'Addresses': ['10.255.0.6/16'],
    'Network': {'CreatedAt': '2017-05-11T11:43:05.297479542Z',
     'DriverState': {'Name': 'overlay',
      'Options': {'com.docker.network.driver.overlay.vxlanid_list': '4096'}},
     'ID': '8fhgsmcv2iy7bcipmkbz3isef',
     'IPAMOptions': {'Configs': [{'Gateway': '10.255.0.1',
        'Subnet': '10.255.0.0/16'}],
      'Driver': {'Name': 'default'}},
     'Spec': {'DriverConfiguration': {},
      'IPAMOptions': {'Configs': [{'Gateway': '10.255.0.1',
         'Subnet': '10.255.0.0/16'}],
       'Driver': {}},
      'Ingress': True,
      'Labels': {},
      'Name': 'ingress'},
     'UpdatedAt': '2017-05-11T11:43:05.312000038Z',
     'Version': {'Index': 6}}}],
  'NodeID': 'hyg7lucp604tvmgns77bo2a0i',
  'ServiceID': 'njmzhx2jigtzclppokkpya0ts',
  'Slot': 1,
  'Spec': {'ContainerSpec': {'Image': 'mjb

In [96]:
print("tasks(containers) running on node swmaster1:" + str(swmaster1.containers.list()) )
print("tasks(containers) running on node swnode1:" + str(swnode1.containers.list()) )
print("tasks(containers) running on node swnode2:" + str(swnode2.containers.list()) )

tasks(containers) running on node swmaster1:[<Container: a357132e3f>]
tasks(containers) running on node swnode1:[]
tasks(containers) running on node swnode2:[]


In [97]:
!docker $(docker-machine config swnode1) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [98]:
!docker $(docker-machine config swnode2) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We have the ip address of the swarm master node which we obtained from docker-machine:

In [100]:
master1ip = m.ip(machine='swmaster1')

Then open your web browser at the page http://<master1ip>:8080 (replace <master1ip>) and you should see a lovely blue whale, as below:

![](images/docker.png)


Alternatively we can obtain a textual representation on the command-line:

In [103]:
!wget -O - http://$master1ip:8080

--2017-05-11 14:07:59--  http://192.168.99.100:8080/
Connecting to 192.168.99.100:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

# TODO: scale service

Now we can scale the service to 3 replicas:

In [ ]:
docker $(docker-machine config swmaster) service scale docker-demo=3

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

# TODO: rolling-update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 20 


In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service update --image mariolet/docker-demo:21 docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

### Verifying the service has been updated

Then open your web browser at the page http://localhost:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)



# TODO:  drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [ ]:
docker $(docker-machine config swmaster) node ls

Let's drain swnode1

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) node update --availability drain swnode1

and now we see that all services on swnode1 are shutdown

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

# TODO:  remove a service

Now let's cleanup by removing our service

In [ ]:
docker $(docker-machine config swmaster) service rm docker-demo

We can check that the service is no longer running:

In [ ]:
docker $(docker-machine config swmaster) service ps docker-demo

In [ ]:
docker $(docker-machine config swmaster) ps

# TODO: deploy a stack

# TODO: working with networks

# TODO: working with volumes

# TODO: deploy a stack